In [1]:
from collections import defaultdict
from glob import glob
import os
from transliterate import translit, get_available_language_codes
from shutil import copy

In [22]:
src = r'C:\Users\m.lavreniuk\Downloads\tasks_new'
dst = r'C:\Users\m.lavreniuk\Downloads\rr'

In [4]:
def get_task_index_map(root, d):
    label_name_dict = {}
    path_to_names = os.path.join(root, d, 'obj.names')
    #with open(path_to_names, 'r') as f:
    with codecs.open(path_to_names, "r", encoding='utf-8') as f:

        names = f.read().splitlines()
    for i, n in enumerate(names):
        label_name_dict[str(i)] = n

    return label_name_dict


def get_pathes_to_txt(root, d):
    num_txt = 0
    for root2, dirs2, files2 in os.walk(os.path.join(root, d, 'obj_train_data')):
        for d2 in dirs2:
            path_to_annotations = os.path.join(root2, d2)
            pathes_to_txt = glob(os.path.join(path_to_annotations, '*.txt'))
            num_txt += len(pathes_to_txt)
            
    return pathes_to_txt, num_txt
    
import codecs


def count_labels(root, d, task_index_map, labels_counter):
    num_objects = 0
    num_fotos = 0
    p, num_txt = get_pathes_to_txt(root, d)
    for t in p:
        with open(t, 'r') as f:
            lines = f.read().splitlines()
            if len(lines) != 0:
                num_fotos += 1
            for l in lines:
                label_id = l.split(' ')[0]
                name = task_index_map[label_id]
                labels_counter[name] += 1
                num_objects += 1
                
    return labels_counter, num_objects, num_fotos, num_txt
                
    
def get_stats(src):
    labels_counter = defaultdict(int)
    task_maps = {}
    for root, dirs, files in os.walk(src, topdown = True):
        for d in dirs:
            index_map = get_task_index_map(root, d)
            task_maps[d] = index_map
            labels_counter, num_objects, num_fotos, num_txt = \
            count_labels(root, d, index_map, labels_counter)            
        break
    return labels_counter, task_maps

In [31]:
counter, task_maps = get_stats(src)
counter

defaultdict(int,
            {'buljon_kur_bol': 501,
             'sup_grech_mal': 425,
             'sup_soljanka_mal': 451,
             'ovosch_zapech': 432,
             'kotl_kur': 395,
             'sal_cesar': 406,
             'adjapsandali': 367,
             'sal_s_tuncom': 360,
             'kapusta_tush': 388,
             'tefteli_mjasn': 390,
             'ljulja_kebab': 968,
             'befstrog_kur': 560,
             'gribi_v_smet': 426,
             'kasha_kukuruz': 422,
             'borsch_mal': 434,
             'bitok_svin': 895,
             'baget_belij': 515,
             'seld_pod_shub': 500,
             'harcho_bol': 434,
             'sal_grech': 500})

In [32]:
import pandas as pd
df = pd.DataFrame.from_dict(counter, orient='index', columns=['count'])
df = df.sort_values(by='count', ascending=False)

In [33]:
k = 150
a = df.loc[df['count'] > 0]

In [34]:
labels = list(a.index.values)

In [35]:
global_label_map = { } 
for i, l in enumerate(labels):
    global_label_map[l] = i

In [16]:
global_label_map = {'kotl_indejka': 9,
 'sup_soljanka_mal': 10,
 'ljulja_kebab': 11,
 'sal_sparj': 12,
 'sup_ovoschnoj_bol': 13,
 'hleb_chernij': 14,
 'bitok_svin': 15,
 'fasol_s_ovoschami': 16,
 'buljon_kur_bol': 17,
 'kartofel_pech': 18,
 'sal_vegan_kinoa': 19,
 'ovosch_gril': 20,
 'ovosch_zapech': 21,
 'sal_morkovka_korejskaja': 22,
 'sup_grech_mal': 23,
 'befstrog_kur': 24,
 'baget_belij': 25,
 'sal_grech': 26,
 'seld_pod_shub': 27,
 'sal_cesar': 28,
 'kotl_kur': 29,
 'kapusta_tush': 30,
 'adjapsandali': 31,
 'tefteli_mjasn': 32,
 'sal_s_tuncom': 33,
 'sup_soljanka_bol': 34,
 'borsch_mal': 2,
 'gribi_v_smet': 35,
 'kasha_kukuruz': 36,
 'harcho_bol': 37}

In [28]:
for i in global_label_map.keys():
    print(i)

file_kur_otvarn
ljulja_kebab
bitok_svin
buljon_kur_bol
sup_borsch_bol
befstrog_kur
baget_grech
ris
sup_soljanka_mal
ovosch_osen
baget_belij
ovosch_zapech
sup_ovosch_mal
sup_grech_mal
sal_grech
seld_pod_shub
sal_cesar
kotl_kur
tefteli_mjasn
kapusta_tush
adjapsandali
sal_s_tuncom
sal_jugnobursk
sup_grech_bol
sal_negnost
gribi_v_smet
harcho_bol
kasha_kukuruz
borsch_mal


In [36]:
global_label_map

{'ljulja_kebab': 0,
 'bitok_svin': 1,
 'befstrog_kur': 2,
 'baget_belij': 3,
 'buljon_kur_bol': 4,
 'seld_pod_shub': 5,
 'sal_grech': 6,
 'sup_soljanka_mal': 7,
 'borsch_mal': 8,
 'harcho_bol': 9,
 'ovosch_zapech': 10,
 'gribi_v_smet': 11,
 'sup_grech_mal': 12,
 'kasha_kukuruz': 13,
 'sal_cesar': 14,
 'kotl_kur': 15,
 'tefteli_mjasn': 16,
 'kapusta_tush': 17,
 'adjapsandali': 18,
 'sal_s_tuncom': 19}

In [16]:
def write_names(label_map):
    tr_names = [translit(key, reversed=True).replace("'", "").replace("SR", "") for key in list(global_label_map.keys())]
    with open(os.path.join(dst, 'obj.names'), 'w') as f:
        [f.write(l + '\n') for l in tr_names]

In [18]:
write_names(global_label_map)

LanguageDetectionError: Can't detect language for the text "kotl_indejka" given.

In [18]:
def convert_lines(lines):
    new_lines = []
    for l in lines:
        l_splited = l.split(' ')
        label_id = l_splited[0]
        name = task_index_map[label_id]
        try:
            new_label_id = global_label_map[name]
        except KeyError as e:
            continue
        else:
            l_splited[0] = str(new_label_id)
            l_new = ' '.join(l_splited)
            new_lines.append(l_new)
            
    return new_lines

In [37]:
skiped_num = 0
path_to_images = os.path.join(dst, 'images')
path_to_labels = os.path.join(dst, 'labels')
for root, dirs, files in os.walk(src, topdown = True):
    for d in dirs:
        task_index_map = task_maps[d] 
        pathes, _ = get_pathes_to_txt(root, d)
        for t in pathes:
            with open(t, 'r') as f:
                lines = f.read().splitlines()
                new_lines = convert_lines(lines)                
            image_path = t.replace(".txt", ".jpg")
            if not os.path.exists(image_path):
                skiped_num += 1
            else:
                pass
                #copy(image_path, path_to_images)
            path_txt = os.path.join(path_to_labels, os.path.basename(t))
            path_txt = t
            with open(path_txt, 'w') as f:
                f.writelines(s + '\n' for s in new_lines)
    break
    
print(f'{skiped_num=}')

skiped_num=0


In [41]:
list(counter.keys())

['sal_vegan_kinoa',
 'sal_morkovka_korejskaja',
 'ovosch_gril',
 'kotl_indejka',
 'sup_soljanka_bol',
 'fasol_s_ovoschami',
 'sal_sparj',
 'kartofel_pech',
 'sup_ovoschnoj_bol',
 'hleb_chernij',
 'kotl_idejka',
 'sup_soljanka_mal']

In [42]:
!pip install openpyxl
df.to_excel('./sku.xlsx')